In [ ]:
import pandas as pd
import gzip
import numpy as np
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from statistics import mean
import time
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler,label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, make_scorer,roc_curve, precision_recall_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Lasso,LinearRegression
import seaborn as sns
from tqdm import tqdm
from xgboost import XGBClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import Ridge
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
import shap
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.feature_selection import f_regression
from sklearn.decomposition import PCA

In [ ]:
# Load the annotation file
annotation_file = 'Human.GRCh38.p13.annot.tsv.gz'

with gzip.open(annotation_file, 'rt') as f:
    annotations = pd.read_csv(f, sep='\t')

# Ensure gene IDs are strings and strip whitespace
annotations['GeneID'] = annotations['GeneID'].astype(str).str.strip()
annotations['Symbol'] = annotations['Symbol'].str.strip()

# Create a dictionary mapping from gene IDs to gene symbols
id_to_symbol = dict(zip(annotations['GeneID'], annotations['Symbol']))

In [ ]:
df_ids = pd.read_csv("GSE234729.csv")
# Set the first column as the index and remove its name
df_ids.set_index(df_ids.columns[0], inplace=True, drop=True)
# Remove the name of the index
df_ids.index.name = None

In [ ]:
# Replace gene IDs with gene symbols in the columns, keeping original IDs if no match is found
df_ids.columns = [id_to_symbol.get(gene, gene) for gene in df_ids.columns]

In [ ]:
# Load the common genes file
common_genes_df = pd.read_csv("common_genes.csv")

# Convert the common genes to a set for easy filtering
common_genes = set(common_genes_df['CommonGenes'])
# Apply the common genes mask to keep only the columns with common gene symbols
df = df_ids.copy()[df_ids.columns.intersection(common_genes)]

### Pre-processing

In [ ]:
# Check for missing values
print("Missing values: ",df.isna().all().sum())
print(df['Pre-eclampsia'].value_counts())

In [ ]:
X = df.iloc[:, :-1]

# Extract the label column (the last column)
y = df.iloc[:, -1]

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Encode the labels into numerical values
y = label_encoder.fit_transform(y)

# Replace the original label column with the encoded labels
df[df.columns[-1]] = y

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
correlation_matrix = X_train.corr()
# get upper triangle of correlation matrix

upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# find features with correlation greater than 0.80

to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]

# drop highly correlated features
X_train.drop(to_drop, axis=1, inplace=True)
X_test.drop(to_drop, axis=1, inplace=True)

### Feature selection

In [ ]:
# Initialize the base estimator
base_estimator = RandomForestClassifier(random_state=42)

# Define the feature subset sizes to use
n_features_list = [10, 20, 30, 40, 50]

# Create dictionaries to store the resulting dataframes
X_train_subsets = {}
X_test_subsets = {}

# Initialize progress bar using tqdm
progress_bar = tqdm(n_features_list, desc="Performing RFE", unit="subset")

# Perform RFE for each feature subset size
for n_features in progress_bar:
    # Record start time
    start_time = time.time()
    
    # Initialize RFE with the base estimator and number of features to select
    rfe = RFE(estimator=base_estimator, n_features_to_select=n_features)
    
    # Fit the RFE model on the training data
    rfe.fit(X_train, y_train)
    
    # Get the selected features' mask (True if the feature is selected, False otherwise)
    selected_features = rfe.support_
    
    # Create new X_train and X_test with the selected features
    X_train_subset = X_train.loc[:, selected_features]
    X_test_subset = X_test.loc[:, selected_features]
    
    # Store the new subsets in the dictionary
    X_train_subsets[f'X_train_{n_features}_features'] = X_train_subset
    X_test_subsets[f'X_test_{n_features}_features'] = X_test_subset
    
    # Calculate time taken
    time_taken = time.time() - start_time
    print(time_taken)
    progress_bar.set_postfix({"Time (s)": round(time_taken, 2)})

# Close progress bar
progress_bar.close()

In [ ]:
import os
# Define directories to store the resulting CSVs
train_save_path = "train_feature_subsets/"
test_save_path = "test_feature_subsets/"
os.makedirs(train_save_path, exist_ok=True)
os.makedirs(test_save_path, exist_ok=True)

# Save all the feature subsets after RFE is done
for n_features in n_features_list:
    # Get the subsets from the dictionaries
    X_train_subset = X_train_subsets[f'X_train_{n_features}_features']
    X_test_subset = X_test_subsets[f'X_test_{n_features}_features']
    
    # Define file names
    train_csv_filename = f"{train_save_path}X_train_{n_features}_features.csv"
    test_csv_filename = f"{test_save_path}X_test_{n_features}_features.csv"
    
    # Save to CSV
    X_train_subset.to_csv(train_csv_filename, index=False)
    X_test_subset.to_csv(test_csv_filename, index=False)

print("All feature subsets have been saved to CSV files.")

### Model development

In [ ]:
# Define the feature subset sizes to use
n_features_list = [10, 20, 30, 40, 50]

# Set directories where train and test CSV files are saved
train_path = "GSE234729_subsets/train_feature_subsets/"
test_path = "GSE234729_subsets/test_feature_subsets/"

# Define parameter grids for each model
param_grids = {
    "RandomForest": {
    "n_estimators": [50, 100, 150, 200],
    "max_depth": [5, 10, 15, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['sqrt', 'log2', None]
    },
    "DecisionTree": {
    "max_depth": [5, 10, 15, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['sqrt', 'log2', None]
    },
    "SVM": {
    "C": [0.1, 1, 10, 100],
    "kernel": ["linear", "rbf", "poly", "sigmoid"],
    "gamma": ["scale", "auto"]
    },
    "XGBoost": {
    "n_estimators": [50, 100, 150],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0]
    },
    "KNN": {
    "n_neighbors": [1],
    "weights": ['uniform', 'distance'],
    "metric": ['euclidean', 'manhattan', 'chebyshev']
    }
}

# Define models for each algorithm
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42),
    "KNN": KNeighborsClassifier()
}

# Dictionary to store trained models and results DataFrame
trained_models = {}
results = []

# Loop over each feature subset and each model
for n_features in n_features_list:
    # Load training and test datasets
    X_train = pd.read_csv(f"{train_path}X_train_{n_features}_features.csv")
    X_test = pd.read_csv(f"{test_path}X_test_{n_features}_features.csv")
    
    
    print("Max value across all features:", X_train.max().max())
    print("Min value across all features:", X_train.min().min())

    print("NaNs:", np.isnan(X_train.values).sum())
    print("Infs:", np.isinf(X_train.values).sum())
    print("Very large (>1e6):", (np.abs(X_train.values) > 1e6).sum())

    for model_name, model in models.items():
        print(f"\nTraining {model_name} model with {n_features} features...")

        # Set up GridSearchCV for the current model
        grid_search = GridSearchCV(
            model, param_grid=param_grids[model_name], cv=5, scoring='accuracy', n_jobs=-1, verbose=1
        )
        grid_search.fit(X_train, y_train)

        # Get the best model and best parameters from GridSearchCV
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

        # Store the model in the dictionary
        trained_models[f"{model_name}_{n_features}_features"] = best_model

        # Evaluate the model on the test set
        y_pred = best_model.predict(X_test)

        # Calculate confusion matrix and metrics
        cm = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

        # Append results to the list
        results.append({
            "Model": model_name,
            "Feature Count": n_features,
            "Best Parameters": best_params,
            "Accuracy": accuracy,
            "Sensitivity (Recall)": sensitivity,
            "Specificity": specificity,
            "Precision": precision,
            "Confusion Matrix": f"TP={tp}, TN={tn}, FP={fp}, FN={fn}"
        })

        # Print results for each feature subset and model
        print(f"Results for {model_name} with {n_features} features:")
        print(f"Best Hyperparameters: {best_params}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Sensitivity (Recall): {sensitivity:.4f}")
        print(f"Specificity: {specificity:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Confusion Matrix: TP={tp}, TN={tn}, FP={fp}, FN={fn}")
        print("-" * 40)
        # Plot the confusion matrix using Seaborn's heatmap
        plt.figure(figsize=(3, 3))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                    xticklabels=[f'Class {i}' for i in range(2)],
                    yticklabels=[f'Class {i}' for i in range(2)],annot_kws={"size": 8, "weight": "bold"})
        plt.title(f'{model_name}',fontweight='bold', fontsize=12)
        plt.xlabel('Predicted', fontweight='bold', fontsize=10)  # Increase font size for x-label
        plt.ylabel('True', fontweight='bold', fontsize=10)  # Increase font size for y-label
        # Set the x and y tick labels to bold and increase their size
        plt.xticks(fontweight='bold', fontsize=10)
        plt.yticks(fontweight='bold', fontsize=10)
        plt.show()

# Convert results to a DataFrame and save to Excel
results_df = pd.DataFrame(results)
results_df.to_excel("model_evaluation_results.xlsx", index=False)
print("\nAll results saved to 'model_evaluation_results.xlsx'")

### Feature ranking

In [ ]:
# Load the common genes from the intersection CSV file
common_genes_df = pd.read_csv("intersection_features.csv")
common_genes = common_genes_df["Common Genes"].tolist()

# Filter the GSE114691 dataset to only include columns from the common genes
gse234729_features = df[common_genes]

# Assume there's a target column named 'target' in GSE114691 dataset
X = gse234729_features
y = df["Pre-eclampsia"]  # Replace 'target' with the actual target column name

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Get feature importances
importances = rf_model.feature_importances_
feature_names = X.columns

# Create a DataFrame with features and their importance scores
feature_importance_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
})

# Sort the features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by="importance", ascending=False)

# Plot the top 10 features
top_10_features = feature_importance_df.head(5)
plt.figure(figsize=(10, 6))
plt.barh(top_10_features["feature"], top_10_features["importance"], color="skyblue")
plt.gca().invert_yaxis()
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.title("Top 10 Important Features in Random Forest")
plt.show()

# Save the ranked features to a CSV
top_10_features.to_csv("GSE234729_feature_importance_ranking.csv", index=False)
print("Feature importance ranking saved to GSE234729_feature_importance_ranking.csv")

### SHAP

In [ ]:
# Select two models from the dictionary
model_1_name = "RandomForest_30_features"  # Replace with your first model name
model_2_name = "XGBoost_50_features"  # Replace with your second model name

# Load the two selected models
model_1 = trained_models[model_1_name]
model_2 = trained_models[model_2_name]

explainer = shap.TreeExplainer(model_2)
shap_values = explainer.shap_values(X_train)

# Define the font properties
title_font_properties = {'fontsize': 20, 'fontweight': 'bold'}
label_font_properties = {'fontsize': 16, 'fontweight': 'bold'}

# Function to format tick labels and adjust spacing
def format_func(value, tick_number):
    return f"{value:.2f}"  # Format to 2 decimal places

plt.title(f"RF SHAP", **title_font_properties)

shap.summary_plot(shap_values[1], X_train, show=False)

plt.xlabel(plt.gca().get_xlabel(), **label_font_properties)
plt.ylabel(plt.gca().get_ylabel(), **label_font_properties)

plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(format_func))
# Customize x and y tick labels
plt.xticks(fontsize=12, fontweight='bold')
plt.yticks(fontsize=12, fontweight='bold')

# Customize color bar label
cbar = plt.gcf().axes[-1]  # Get the color bar
cbar.set_ylabel('Feature value', fontsize=16, fontweight='bold')

# Customize color bar tick labels
cbar.set_yticklabels(['Low', 'High'], fontsize=12, fontweight='bold')

plt.tight_layout()

plt.show()